In [80]:
import nibabel as nib
from nilearn import image
from nilearn import plotting
from nilearn import datasets
from nilearn import maskers
from nilearn.connectome import ConnectivityMeasure
from tqdm import tqdm
import numpy as np
import os
import re
import pickle
from nilearn import datasets
from nilearn.input_data import NiftiLabelsMasker
import nibabel as nib
import numpy as np

In [46]:
# Fetch the AAL atlas
aal_atlas = datasets.fetch_atlas_aal()

# Get the number of regions (ROIs) from AAL atlas
n_regions = len(aal_atlas['labels'])

print(f'AAL atlas has {n_regions} ROIs.')

# Print the first few region names
print(f"Regions in AAL atlas: {aal_atlas['labels'][:]}, {len(aal_atlas['labels'][:])}")

AAL atlas has 116 ROIs.
Regions in AAL atlas: ['Precentral_L', 'Precentral_R', 'Frontal_Sup_L', 'Frontal_Sup_R', 'Frontal_Sup_Orb_L', 'Frontal_Sup_Orb_R', 'Frontal_Mid_L', 'Frontal_Mid_R', 'Frontal_Mid_Orb_L', 'Frontal_Mid_Orb_R', 'Frontal_Inf_Oper_L', 'Frontal_Inf_Oper_R', 'Frontal_Inf_Tri_L', 'Frontal_Inf_Tri_R', 'Frontal_Inf_Orb_L', 'Frontal_Inf_Orb_R', 'Rolandic_Oper_L', 'Rolandic_Oper_R', 'Supp_Motor_Area_L', 'Supp_Motor_Area_R', 'Olfactory_L', 'Olfactory_R', 'Frontal_Sup_Medial_L', 'Frontal_Sup_Medial_R', 'Frontal_Med_Orb_L', 'Frontal_Med_Orb_R', 'Rectus_L', 'Rectus_R', 'Insula_L', 'Insula_R', 'Cingulum_Ant_L', 'Cingulum_Ant_R', 'Cingulum_Mid_L', 'Cingulum_Mid_R', 'Cingulum_Post_L', 'Cingulum_Post_R', 'Hippocampus_L', 'Hippocampus_R', 'ParaHippocampal_L', 'ParaHippocampal_R', 'Amygdala_L', 'Amygdala_R', 'Calcarine_L', 'Calcarine_R', 'Cuneus_L', 'Cuneus_R', 'Lingual_L', 'Lingual_R', 'Occipital_Sup_L', 'Occipital_Sup_R', 'Occipital_Mid_L', 'Occipital_Mid_R', 'Occipital_Inf_L', 'Occ

In [49]:
masker = maskers.NiftiLabelsMasker(
    labels_img=aal_atlas.maps,  # Use maps_img instead of maps
    resampling_target="data",
    t_r=3, 
    detrend=True,
    low_pass=0.1, 
    high_pass=0.01
).fit()


In [50]:
correlation_measure = ConnectivityMeasure(kind='correlation')

In [11]:
img_dir = r"C:\Users\praneeth darlanka\Desktop\ALL_AD\003_S_6833\Struct\003_S_6833.nii"

In [15]:
first_image = nib.load(img_dir)

In [20]:
img_data = first_image.get_fdata()

In [23]:
main_dir = r"C:\Users\praneeth darlanka\Desktop\ALL_AD"

In [54]:
func_features = []

In [55]:
cnt = 0
for sub in os.listdir(main_dir):
    subject_path = os.path.join(main_dir,sub)
    for type in os.listdir(subject_path):
        if(type == "Func"):
            struct_dir = os.path.join(subject_path,type)
            file = os.listdir(struct_dir)
            for f in file:
                if f.startswith('swar'):
                    cnt += 1
                    lovda_img = os.path.join(struct_dir,f)
                    fmri_img = nib.load(lovda_img)
                    img = image.load_img(fmri_img)
                    features = masker.fit_transform(fmri_img)
                    correlation_matrix = correlation_measure.fit_transform([features])[0]
                    func_features.append(correlation_matrix.reshape(-1))
                    print(cnt)
                    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39


In [73]:
yoo  = r"C:\Users\praneeth darlanka\Desktop\022_S_6013_62.57_F\Func\swar022_S_6013_62.57_F.nii"
fmri_img = nib.load(yoo)
img = image.load_img(fmri_img)
features = masker.fit_transform(fmri_img)
correlation_matrix = correlation_measure.fit_transform([features])[0]
func_features.append(correlation_matrix.reshape(-1))

In [76]:
len(func_features[0])

13456

In [97]:
struc_features = []
y = []

In [99]:
len(func_features)

40

In [98]:
lovda_img = r'C:\Users\praneeth darlanka\Desktop\022_S_6013_62.57_F\Struct\022_S_6013_62.57_F.nii'
fmri_img = nib.load(lovda_img)
img = image.load_img(fmri_img)
features = masker.fit_transform(fmri_img)
correlation_matrix = correlation_measure.fit_transform([features])[0]
y.append(correlation_matrix.reshape(-1))

ValueError: The length of the input vector x must be greater than padlen, which is 33.

In [89]:
correlation_matrix.shape

(116, 116)

In [134]:
import pickle
pickle.dump(func_features, open(r'C:\Users\praneeth darlanka\Desktop\features\func_features', 'wb'))


In [92]:
loading_feature1 = pickle.load(open(r"C:\Users\praneeth darlanka\Desktop\features\func_features.pkl", 'rb'))

In [93]:
loading_feature = np.array(loading_feature1)

In [94]:
loading_feature.shape

(40, 13456)

In [100]:
aal_atlas = datasets.fetch_atlas_aal()

In [101]:
struct_masker = NiftiLabelsMasker(
    labels_img=aal_atlas.maps,  # ROI map from the AAL atlas
    standardize=True,           # Standardize data (mean=0, variance=1)
    smoothing_fwhm=4            # Apply spatial smoothing (optional, adjust as needed)
).fit()

In [120]:
struct_features = []

In [122]:
cnt = 0
for sub in os.listdir(main_dir):
    subject_path = os.path.join(main_dir,sub)
    for type in os.listdir(subject_path):
        if(type == "Struct"):
            struct_dir = os.path.join(subject_path,type)
            file = os.listdir(struct_dir)
            cnt += 1
            f = file[0]
            lovda_img = os.path.join(struct_dir,f)
            struct_image = nib.load(lovda_img)
            features = struct_masker.transform(struct_image)
            correlation_matrix = correlation_measure.fit_transform([features])[0]
            struct_features.append(correlation_matrix.reshape(-1))
            print(cnt,f)

C:\Users\praneeth darlanka\anaconda3\Lib\site-packages\nilearn\signal.py:74: UserWarning: Standardization of 3D signal has been requested but would lead to zero values. Skipping.
  warnings.warn(
C:\Users\praneeth darlanka\anaconda3\Lib\site-packages\sklearn\covariance\_shrunk_covariance.py:327: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(
C:\Users\praneeth darlanka\anaconda3\Lib\site-packages\sklearn\covariance\_empirical_covariance.py:95: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(
C:\Users\praneeth darlanka\anaconda3\Lib\site-packages\nilearn\connectome\connectivity_matrices.py:345: RuntimeWarning: divide by zero encountered in divide
  diagonal = np.atleast_2d(1.0 / np.sqrt(np.diag(covariance)))
C:\Users\praneeth darlanka\anaconda3\Lib\site-packages\nilearn\connectome\connectivity_matrices.py:346: RuntimeWarning: invalid value encountered in multiply
  correlation = covariance *

1 003_S_6264.nii


C:\Users\praneeth darlanka\anaconda3\Lib\site-packages\nilearn\signal.py:74: UserWarning: Standardization of 3D signal has been requested but would lead to zero values. Skipping.
  warnings.warn(
C:\Users\praneeth darlanka\anaconda3\Lib\site-packages\sklearn\covariance\_shrunk_covariance.py:327: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(
C:\Users\praneeth darlanka\anaconda3\Lib\site-packages\sklearn\covariance\_empirical_covariance.py:95: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(
C:\Users\praneeth darlanka\anaconda3\Lib\site-packages\nilearn\connectome\connectivity_matrices.py:345: RuntimeWarning: divide by zero encountered in divide
  diagonal = np.atleast_2d(1.0 / np.sqrt(np.diag(covariance)))
C:\Users\praneeth darlanka\anaconda3\Lib\site-packages\nilearn\connectome\connectivity_matrices.py:346: RuntimeWarning: invalid value encountered in multiply
  correlation = covariance *

2 003_S_6833.nii


C:\Users\praneeth darlanka\anaconda3\Lib\site-packages\nilearn\signal.py:74: UserWarning: Standardization of 3D signal has been requested but would lead to zero values. Skipping.
  warnings.warn(
C:\Users\praneeth darlanka\anaconda3\Lib\site-packages\sklearn\covariance\_shrunk_covariance.py:327: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(
C:\Users\praneeth darlanka\anaconda3\Lib\site-packages\sklearn\covariance\_empirical_covariance.py:95: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(
C:\Users\praneeth darlanka\anaconda3\Lib\site-packages\nilearn\connectome\connectivity_matrices.py:345: RuntimeWarning: divide by zero encountered in divide
  diagonal = np.atleast_2d(1.0 / np.sqrt(np.diag(covariance)))
C:\Users\praneeth darlanka\anaconda3\Lib\site-packages\nilearn\connectome\connectivity_matrices.py:346: RuntimeWarning: invalid value encountered in multiply
  correlation = covariance *

...

C:\Users\praneeth darlanka\anaconda3\Lib\site-packages\nilearn\signal.py:74: UserWarning: Standardization of 3D signal has been requested but would lead to zero values. Skipping.
  warnings.warn(
C:\Users\praneeth darlanka\anaconda3\Lib\site-packages\sklearn\covariance\_shrunk_covariance.py:327: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(
C:\Users\praneeth darlanka\anaconda3\Lib\site-packages\sklearn\covariance\_empirical_covariance.py:95: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(
C:\Users\praneeth darlanka\anaconda3\Lib\site-packages\nilearn\connectome\connectivity_matrices.py:345: RuntimeWarning: divide by zero encountered in divide
  diagonal = np.atleast_2d(1.0 / np.sqrt(np.diag(covariance)))
C:\Users\praneeth darlanka\anaconda3\Lib\site-packages\nilearn\connectome\connectivity_matrices.py:346: RuntimeWarning: invalid value encountered in multiply
  correlation = covariance *

37 168_S_6921_76.55_M.nii


C:\Users\praneeth darlanka\anaconda3\Lib\site-packages\nilearn\signal.py:74: UserWarning: Standardization of 3D signal has been requested but would lead to zero values. Skipping.
  warnings.warn(
C:\Users\praneeth darlanka\anaconda3\Lib\site-packages\sklearn\covariance\_shrunk_covariance.py:327: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(
C:\Users\praneeth darlanka\anaconda3\Lib\site-packages\sklearn\covariance\_empirical_covariance.py:95: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(
C:\Users\praneeth darlanka\anaconda3\Lib\site-packages\nilearn\connectome\connectivity_matrices.py:345: RuntimeWarning: divide by zero encountered in divide
  diagonal = np.atleast_2d(1.0 / np.sqrt(np.diag(covariance)))
C:\Users\praneeth darlanka\anaconda3\Lib\site-packages\nilearn\connectome\connectivity_matrices.py:346: RuntimeWarning: invalid value encountered in multiply
  correlation = covariance *

38 168_S_6921_78.01_M.nii
39 168_S_6938.nii


C:\Users\praneeth darlanka\anaconda3\Lib\site-packages\nilearn\signal.py:74: UserWarning: Standardization of 3D signal has been requested but would lead to zero values. Skipping.
  warnings.warn(
C:\Users\praneeth darlanka\anaconda3\Lib\site-packages\sklearn\covariance\_shrunk_covariance.py:327: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(
C:\Users\praneeth darlanka\anaconda3\Lib\site-packages\sklearn\covariance\_empirical_covariance.py:95: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(
C:\Users\praneeth darlanka\anaconda3\Lib\site-packages\nilearn\connectome\connectivity_matrices.py:345: RuntimeWarning: divide by zero encountered in divide
  diagonal = np.atleast_2d(1.0 / np.sqrt(np.diag(covariance)))
C:\Users\praneeth darlanka\anaconda3\Lib\site-packages\nilearn\connectome\connectivity_matrices.py:346: RuntimeWarning: invalid value encountered in multiply
  correlation = covariance *

In [124]:
lovda_img = r'C:\Users\praneeth darlanka\Desktop\022_S_6013_62.57_F\Struct\022_S_6013_62.57_F.nii'
struct_image = nib.load(lovda_img)
features = struct_masker.transform(struct_image)
correlation_matrix = correlation_measure.fit_transform([features])[0]
struct_features.append(correlation_matrix.reshape(-1))


39 168_S_6938.nii


C:\Users\praneeth darlanka\anaconda3\Lib\site-packages\nilearn\signal.py:74: UserWarning: Standardization of 3D signal has been requested but would lead to zero values. Skipping.
  warnings.warn(
C:\Users\praneeth darlanka\anaconda3\Lib\site-packages\sklearn\covariance\_shrunk_covariance.py:327: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(
C:\Users\praneeth darlanka\anaconda3\Lib\site-packages\sklearn\covariance\_empirical_covariance.py:95: UserWarning: Only one sample available. You may want to reshape your data array
  warnings.warn(
C:\Users\praneeth darlanka\anaconda3\Lib\site-packages\nilearn\connectome\connectivity_matrices.py:345: RuntimeWarning: divide by zero encountered in divide
  diagonal = np.atleast_2d(1.0 / np.sqrt(np.diag(covariance)))
C:\Users\praneeth darlanka\anaconda3\Lib\site-packages\nilearn\connectome\connectivity_matrices.py:346: RuntimeWarning: invalid value encountered in multiply
  correlation = covariance *

In [126]:
len(struct_features)

40

In [127]:
func = np.array(func_features)

In [128]:
struc = np.array(struct_features)

In [129]:
struc.shape

(40, 13456)

In [130]:
func.shape

(40, 13456)

In [131]:
combined_features = np.hstack((func, struc))

In [133]:
combined_features.shape

(40, 26912)

In [135]:
pickle.dump(combined_features, open(r'C:\Users\praneeth darlanka\Desktop\features\combined_features', 'wb'))

In [136]:
pickle.dump(struct_features, open(r'C:\Users\praneeth darlanka\Desktop\features\struct_features', 'wb'))
